# Flight Delay Prediction

## Data Loading and Preprocessing

In [ ]:
import pandas as pd

In [ ]:
# Load the cleaned flight data
flights_df = pd.read_parquet("cleaned_flights.parquet")

In [ ]:
# Extract the hour from the 'DATE' column and create a new column 'DEPARTURE_HOUR'
flights_df['DEPARTURE_HOUR'] = flights_df['DATE'].dt.hour

In [ ]:
# Replace missing values in 'DAILY_SNOWFALL' with 0
flights_df['DAILY_SNOWFALL'] = flights_df['DAILY_SNOWFALL'].fillna(0)

In [ ]:
# Select only delayed flights from flights_df
delayed_flights = flights_df[flights_df['ARRIVAL_DELAY'] > 0].copy()

## MLX Neural Network Model

In [ ]:
import numpy as np
import mlx.core as mx
import mlx.nn as nn
import mlx.optimizers as optim
from sklearn.metrics import mean_squared_error, r2_score

In [ ]:
# Standardize numeric columns for sklearn models
for col in ['MONTH', 'DEPARTURE_HOUR', 'DAY_OF_WEEK', 'DISTANCE']:
    col_mean = delayed_flights[col].mean()
    col_std  = delayed_flights[col].std()
    delayed_flights[col] = (delayed_flights[col] - col_mean) / col_std

In [ ]:
# Extract features for sklearn models
numeric_feats = delayed_flights[['MONTH', 'DEPARTURE_HOUR', 'DAY_OF_WEEK', 'DISTANCE', 'DAILY_SNOWFALL']].values
categorical_feats = pd.get_dummies(delayed_flights[['AIRLINE', 'origin_airport/AIRPORT', 'destination_airport/AIRPORT']]).values
X = np.hstack([numeric_feats, categorical_feats])
y = delayed_flights['ARRIVAL_DELAY'].values

In [ ]:
split = int(0.8 * X.shape[0])
X_train, X_test = X[:split], X[split:]
y_train, y_test = y[:split], y[split:]

In [ ]:
# Convert data to MLX arrays
X_train_mx = mx.array(X_train, dtype=mx.float32)
y_train_mx = mx.array(y_train, dtype=mx.float32).reshape(-1, 1)
X_test_mx  = mx.array(X_test, dtype=mx.float32)
y_test_mx  = mx.array(y_test, dtype=mx.float32).reshape(-1, 1)

In [ ]:
# Define the Feedforward Model
class FlightDelayPredictor(nn.Module):
    def __init__(self, input_dim, hidden_dim1=128, hidden_dim2=64, output_dim=1):
        super().__init__()
        self.layer1 = nn.Linear(input_dim, hidden_dim1)
        self.layer2 = nn.Linear(hidden_dim1, hidden_dim2)
        self.layer3 = nn.Linear(hidden_dim2, output_dim)

    def __call__(self, x):
        x = mx.maximum(self.layer1(x), 0) # ReLU activation
        x = mx.maximum(self.layer2(x), 0) # ReLU activation
        x = self.layer3(x)
        return x

# Instantiate the model
input_features = X_train_mx.shape[1]
model = FlightDelayPredictor(input_dim=input_features)
mx.eval(model.parameters()) # Initialize parameters

In [ ]:
# Define Loss and Gradient Function
def mse_loss(model, x, y):
    return mx.mean(mx.square(model(x) - y))

# Get the function that calculates loss and gradients
loss_and_grad_fn = nn.value_and_grad(model, mse_loss)

In [ ]:
# Setup Optimizer
learning_rate = 1e-4
optimizer = optim.Adam(learning_rate=learning_rate)

In [ ]:
# Training Loop
num_epochs = 20
batch_size = 256
num_samples = X_train_mx.shape[0]

for epoch in range(num_epochs):
    epoch_loss = 0.0
    # Create batches
    permutation = mx.array(np.random.permutation(num_samples))
    for i in range(0, num_samples, batch_size):
        batch_indices = permutation[i:i+batch_size]
        X_batch = X_train_mx[batch_indices]
        y_batch = y_train_mx[batch_indices]

        # Calculate loss and gradients for the batch
        loss, grads = loss_and_grad_fn(model, X_batch, y_batch)

        # Update model parameters and optimizer state
        optimizer.update(model, grads)

        # Force evaluation
        mx.eval(model.parameters(), optimizer.state)

        epoch_loss += loss.item() * X_batch.shape[0]

    avg_epoch_loss = epoch_loss / num_samples
    print(f"Epoch {epoch+1}/{num_epochs}, Average Loss: {avg_epoch_loss:.4f}")

In [ ]:
# Get predictions (ensure model is in eval mode if using dropout/batchnorm, though not used here)
test_predictions_mx = model(X_test_mx)
test_loss = mse_loss(model, X_test_mx, y_test_mx)
mx.eval(test_predictions_mx, test_loss) # Evaluate predictions and loss

print(f"Test MSE Loss: {test_loss.item():.4f}")

# Print RMSE
rmse = np.sqrt(test_loss.item())
print(f"Test RMSE: {rmse:.4f}")

# Optional: Calculate R2 score using sklearn
test_predictions_np = np.array(test_predictions_mx)
y_test_np = np.array(y_test_mx)
r2 = r2_score(y_test_np, test_predictions_np)
print(f"Test R2 Score: {r2:.4f}")